In [1]:
#Q1. You are work#ng on a mach#ne learn#ng project where you have a dataset conta#n#ng numer#cal and
#categor#cal features. You have #dent#f#ed that some of the features are h#ghly correlated and there are
#m#ss#ng values #n some of the columns. You want to bu#ld a p#pel#ne that automates the feature
#eng#neer#ng process and handles the m#ss#ng values?

#Design a p#pel#ne that #ncludes the follow#ng steps"
#Use an automated feature select#on method to #dent#fy the #mportant features #n the datasetC
#Create a numer#cal p#pel#ne that #ncludes the follow#ng steps"
#Impute the m#ss#ng values #n the numer#cal columns us#ng the mean of the column valuesC
#Scale the numer#cal columns us#ng standard#sat#onC
#Create a categor#cal p#pel#ne that #ncludes the follow#ng steps"
#Impute the m#ss#ng values #n the categor#cal columns us#ng the most frequent value of the columnC
#One-hot encode the categor#cal columnsC
#Comb#ne the numer#cal and categor#cal p#pel#nes us#ng a ColumnTransformerC
#Use a Random Forest Class#f#er to bu#ld the f#nal modelC
#Evaluate the accuracy of the model on the test datasetD

## Stage 1 Loading,Cleaning and Encoding Data

In [55]:
# Import the necessary libraries and load the dataset
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from warnings import filterwarnings
from sklearn.metrics import RocCurveDisplay
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

filterwarnings('ignore')


df=pd.read_csv('winequality-red.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [56]:
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [57]:
df.duplicated().sum()

240

In [58]:
df.drop_duplicates(inplace=True)

In [59]:
df.duplicated().sum()

0

In [60]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('quality', axis=1), df['quality'], test_size=0.3, random_state=42)
X_train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
374,14.0,0.41,0.63,3.8,0.089,6.0,47.0,1.00140,3.01,0.81,10.8
1225,9.2,0.46,0.23,2.6,0.091,18.0,77.0,0.99922,3.15,0.51,9.4
363,12.5,0.46,0.63,2.0,0.071,6.0,15.0,0.99880,2.99,0.87,10.2
1426,7.7,0.23,0.37,1.8,0.046,23.0,60.0,0.99710,3.41,0.71,12.1
89,7.0,0.62,0.08,1.8,0.076,8.0,24.0,0.99780,3.48,0.53,9.0
...,...,...,...,...,...,...,...,...,...,...,...
1285,11.3,0.37,0.50,1.8,0.090,20.0,47.0,0.99734,3.15,0.57,10.5
1329,7.4,0.60,0.26,2.1,0.083,17.0,91.0,0.99616,3.29,0.56,9.8
1526,6.8,0.47,0.08,2.2,0.064,18.0,38.0,0.99553,3.30,0.65,9.6
1011,8.9,0.32,0.31,2.0,0.088,12.0,19.0,0.99570,3.17,0.55,10.4


In [61]:
# Define the numerical pipeline
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
num_pipeline

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler())])

In [62]:

# Define the categorical pipeline
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
   ('encoder', OneHotEncoder())
])
cat_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder', OneHotEncoder())])

In [71]:

# Combine the numerical and categorical pipelines using a ColumnTransformer


preprocessor = ColumnTransformer([
    ('num', num_pipeline, X_train.select_dtypes(exclude="object").columns),
    ('cat', cat_pipeline,  X_train.select_dtypes(include="object").columns)
])
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol'],
      dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder', OneHotEncoder())]),
                                 Index([], dtype='object'))])

In [72]:
# Use an automated feature selection method to identify the important features in the dataset.
selector = SelectKBest(score_func=f_regression, k=5)
selector

SelectKBest(k=5, score_func=<function f_regression at 0x7f47a5461bd0>)

In [73]:
# Use a Random Forest Classifier to build the final model.
rfc = RandomForestClassifier(random_state=42)
rfc

RandomForestClassifier(random_state=42)

In [74]:
# Define the pipeline that includes all the steps.
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('selector', selector),
    ('classifier', rfc)
])
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index([], dtype='object'))])),
                ('selector',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x7f47a5461bd0>)),
                ('classifier', RandomForestClassifier(random_state=42))])

In [75]:
# Fit the pipeline on the training data.
pipeline.fit(X_train, y_train)

# Evaluate the accuracy of the model on the test dataset.
score = pipeline.score(X_test, y_test)
score

0.5931372549019608

The accuracy of the pipeline is 59%

In [ ]:
#Q2. Bu#ld a p#pel#ne that #ncludes a random forest class#f#er and a log#st#c regress#on class#f#er, and then
#use a vot#ng class#f#er to comb#ne the#r pred#ct#ons. Tra#n the p#pel#ne on the #r#s dataset and evaluate #ts
#accuracy.

In [2]:
#import libraries and load dataset

from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression

iris = load_iris()
X = iris.data[:, :2]
y = iris.target

In [3]:
clf1 = RandomForestClassifier(n_estimators=50, random_state=1)
clf2 = LogisticRegression(random_state=1)

eclf = VotingClassifier(estimators=[('rf', clf1), ('lr', clf2)], voting='soft')
eclf.fit(X, y)

print(eclf.score(X, y))

0.8866666666666667
